In [ ]:
##########################################################################################################################################
######################## This script extracts 55 covariates values at each of the soil algae data point position #########################
##########################################################################################################################################

In [1]:
import ee # Import the Python API package
ee.Authenticate() # Authenticate to the Earth Engine servers
ee.Initialize() # Initialize the API

Enter verification code:  4/1AX4XfWiTsXEThSEhjvC_7_XoAtq2gINsN5a2ATUKAPaHLVt_r4QllwxPXoQ



Successfully saved authorization token.


In [2]:
import geemap # The geemap Python package is built upon the ipyleaflet and folium packages and implements several methods for interacting with Earth Engine data layer

In [27]:
# Create a map
Map1 = geemap.Map(center=[0,0], zoom=2)
Map1

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Toggl…

In [4]:
# Loading sampling points stored in personnal repository
ProductivityPts = ee.FeatureCollection('users/romainwalcker/Phototrophs_Productivity_AllPts')
print('Size of the productivity data collection',ProductivityPts.size().getInfo())

Size of the productivity data collection 102


In [28]:
# Add points to the map
Map1.addLayer(ProductivityPts, {},'Sampling points')

In [6]:
# Loading environmental variables (Table S3 of the paper)
# Climate variables
WorldClim = ee.Image('WORLDCLIM/V1/BIO');
bio01 = WorldClim.select('bio01')# Annual Mean Temperature
bio02 = WorldClim.select('bio02')# Mean Diurnal Range
bio03 = WorldClim.select('bio03')# Isothermality
bio04 = WorldClim.select('bio04')# Temperature Seasonality
bio05 = WorldClim.select('bio05')# Max Temperature of Warmest Month
bio06 = WorldClim.select('bio06')# Min Temperature of Coldest Month
bio07 = WorldClim.select('bio07')# Temperature Annual Range (BIO5-BIO6)
bio08 = WorldClim.select('bio08')# Mean Temperature of Wettest Quarter
bio09 = WorldClim.select('bio09')# Mean Temperature of Driest Quarter
bio10 = WorldClim.select('bio10')# Mean Temperature of Warmest Quarter
bio11 = WorldClim.select('bio11')# Mean Temperature of Coldest Quarter
bio12 = WorldClim.select('bio12')# Annual Precipitation
bio13 = WorldClim.select('bio13')# Precipitation of Wettest Month
bio14 = WorldClim.select('bio14')# Precipitation of Driest Month
bio15 = WorldClim.select('bio15')# Precipitation Seasonality
bio16 = WorldClim.select('bio16')# Precipitation of Wettest Quarter
bio17 = WorldClim.select('bio17')# Precipitation of Driest Quarter
bio18 = WorldClim.select('bio18')# Precipitation of Warmest Quarter
bio19 = WorldClim.select('bio19')# Precipitation of Coldest Quarter

TerraClim = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
aet = TerraClim.select('aet').mean()# Actual evapotranspiration
tdef = TerraClim.select('def').mean()# Climate water deficit
pdsi = TerraClim.select('pdsi').mean()# Palmer Drought Severity Index
pet = TerraClim.select('pet').mean()# Reference evapotranspiration
pr = TerraClim.select('pr').mean()# Precipitation accumulation
ro = TerraClim.select('ro').mean()# Runoff 
srad = TerraClim.select('srad').mean()# Downward surface shortwave radiation
swe = TerraClim.select('swe').mean()# Snow water equivalent
tmmn = TerraClim.select('tmmn').mean()# Minimum temperature
tmmx = TerraClim.select('tmmx').mean()# Maximum temperature
vap = TerraClim.select('vap').mean()# Vapor pressure
vpd = TerraClim.select('vpd').mean()# Vapor pressure deficit
wspd = TerraClim.select('vs').mean().rename('wspd')# Wind-speed at 10m

In [7]:
# Soil variables
USDA4A1H_M = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02");
bulk = USDA4A1H_M.select('b0').unmask().rename('bulk')# Soil bulk density at 0 cm depth
USDA3A1A1A_M = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02");
clay = USDA3A1A1A_M.select('b0').rename('clay')# Clay content at 0 cm depth
USDA6A1C_M = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02");
soc = USDA6A1C_M.select('b0').rename('soc')# Soil organic carbon content at 0 cm depth
USDA4C1A2A_M = ee.Image("OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02");
ph = USDA4C1A2A_M.select('b0').rename('ph')# Soil pH in H2O at 0 cm depth
USDATT_M = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02");
text = USDATT_M.select('b0').rename('text')# Soil texture classes
USDA3A1A1A_M = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02");
sand = USDA3A1A1A_M.select('b0').rename('sand')# Sand content
USDA4B1C_M = ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01");
water = USDA4B1C_M.select('b0').rename('water')# Soil water content

TerraClim = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE');
moist = TerraClim.select('soil').mean().rename('moist')# Soil moisture, derived using a one-dimensional soil water balance model

SMAP = ee.ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture');
ssm = SMAP.select('ssm').mean()# Surface soil moisture
susm = SMAP.select('susm').mean()# Subsurface soil moisture
smp = SMAP.select('smp').mean()# Soil moisture profile
ssma = SMAP.select('ssma').mean()# Surface soil moisture anomaly
susma = SMAP.select('susma').mean()# Subsurface soil moisture anomaly

In [8]:
# Topographic variables
GMTED = ee.Image('USGS/GMTED2010')
height = GMTED.select('be75').rename('height')# Elevation
  
Coordonates = ee.Image.pixelLonLat()
lon = Coordonates.select('longitude').rename('lon')
lat = Coordonates.select('latitude').rename('lat')

In [9]:
# Vegetation variables
MOD13Q1 = ee.ImageCollection('MODIS/006/MOD13Q1')
ndvi = MOD13Q1.select('NDVI').mean().rename('ndvi')# Normalized Difference Vegetation Index
evi = MOD13Q1.select('EVI').mean().rename('evi')# Enhanced Vegetation Index
  
MOD17A2H = ee.ImageCollection('MODIS/006/MOD17A2H')
gpp = MOD17A2H.select('Gpp').mean().rename('gpp')# Gross primary production
npp = MOD17A2H.select('PsnNet').mean().rename('npp')# Net photosynthesis
  
MCD15A3H = ee.ImageCollection('MODIS/006/MCD15A3H');
lai = MCD15A3H.select('Lai').mean().rename('lai')# leaf area index

biomes = ee.Image('OpenLandMap/PNV/PNV_BIOME-TYPE_BIOME00K_C/v01').rename('biomes')# Potential distribution of biomes

In [10]:
# Human variables
GPW = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Density").first()
pop = GPW.select('population_density').rename('pop')# The estimated number of persons per square kilometer

In [36]:
# Stacking environmental variable in a multiband image
compositeOfInterest = biomes.addBands(bio01).addBands(bio02).addBands(bio03).addBands(bio04).addBands(bio05).addBands(bio06).addBands(bio07).addBands(bio08).addBands(bio09).addBands(bio10).addBands(bio11).addBands(bio12).addBands(bio13).addBands(bio14).addBands(bio15).addBands(bio16).addBands(bio17).addBands(bio18).addBands(bio19).addBands(aet).addBands(tdef).addBands(pdsi).addBands(pet).addBands(pr).addBands(ro).addBands(srad).addBands(swe).addBands(tmmn).addBands(tmmx).addBands(vap).addBands(vpd).addBands(wspd).addBands(ndvi).addBands(evi).addBands(gpp).addBands(npp).addBands(lai).addBands(bulk).addBands(clay).addBands(soc).addBands(ph).addBands(text).addBands(sand).addBands(water).addBands(moist).addBands(ssm).addBands(susm).addBands(smp).addBands(ssma).addBands(susma).addBands(height).addBands(lat).addBands(lon).addBands(pop)

In [51]:
# Sampling
# The code below allows for sampling the collection without dropping missing values for variables reduction
ProductivityPtsSampled = compositeOfInterest.reduceRegions(reducer= ee.Reducer.first(),collection= ProductivityPts,scale= compositeOfInterest.select('soc').projection().nominalScale().getInfo())
print('Size of the sampled data collection',ProductivityPtsSampled.size().getInfo())
print('First sample', ProductivityPtsSampled.limit(1).getInfo());

Size of the sampled data collection 102
First sample {'type': 'FeatureCollection', 'columns': {'Group': 'String', 'ID': 'Float', 'aet': 'Float<-32768.0, 32767.0>', 'bio01': 'Float', 'bio02': 'Float', 'bio03': 'Float', 'bio04': 'Float', 'bio05': 'Float', 'bio06': 'Float', 'bio07': 'Float', 'bio08': 'Float', 'bio09': 'Float', 'bio10': 'Float', 'bio11': 'Float', 'bio12': 'Float', 'bio13': 'Float', 'bio14': 'Float', 'bio15': 'Float', 'bio16': 'Float', 'bio17': 'Float', 'bio18': 'Float', 'bio19': 'Float', 'biomes': 'Short<0, 255>', 'bulk': 'Short<0, 255>', 'clay': 'Short<0, 255>', 'def': 'Float<-32768.0, 32767.0>', 'evi': 'Float<-32768.0, 32767.0>', 'gpp': 'Float<-32768.0, 32767.0>', 'height': 'Short', 'lai': 'Float<0.0, 255.0>', 'lat': 'Float', 'log_primpr': 'Float', 'lon': 'Float', 'moist': 'Float<-32768.0, 32767.0>', 'ndvi': 'Float<-32768.0, 32767.0>', 'npp': 'Float<-32768.0, 32767.0>', 'pdsi': 'Float<-32768.0, 32767.0>', 'pet': 'Float<-32768.0, 32767.0>', 'ph': 'Short<0, 255>', 'pop': '